- 카카오 API를 이용해 주소에서 위경도를 통해 주소를 추출해보자.우선, 카카오에서 제공하는 API를 사용하기 위해 Kakao Developers에서 api키를 발급 받아야 한다. API key를 발급받자. **REST API키**

<center>
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FUNRpG%2Fbtq4ct5TUpE%2FnSYkkK7AXtoR6EKsbkUKLK%2Fimg.png" width="600" height="400"><br>
</center>

- 카카오 Developer 주소: [Kakao Developers](https://developers.kakao.com/console/app)
- 참고: [미완성의신](https://unfinishedgod.netlify.app/2021/04/28/python-%EC%B9%B4%EC%B9%B4%EC%98%A4-api%EB%A5%BC-%EC%82%AC%EC%9A%A9%ED%95%9C-%EC%A3%BC%EC%86%8C-%EC%9C%84%EA%B2%BD%EB%8F%84-%EC%B6%94%EC%B6%9C/)

먼저 필요한 모듈을 import 한다. requests는 HTTP를 호출할 때 사용하는 라이브러리이다.

In [110]:
import json
import requests
import folium
import pandas as pd

#### 1. API 셋팅하기

이곳에서 사용할 API키는 REST API 키를 참고 하도록 하자. 이번에 사용할 API이다.

In [120]:
api_key = 'a6fcb08aec1b4dc8cf2869322294ab95'

#### 2. 지리정보 API를 사용해 주소에서 위경도에서 추출
이제 GET()함수를 통해 다음과 같이 API를 호출 해주자. 주소는 남산을 택했으며 "남산" 주소를 검색 결과 “서울 중구 회현동1가”로 나오게 되어 이곳의 위경도를 추출하려고 한다. API문서에서는 다음에 위치 한다.

- 개발가이드: [주소 검색](https://developers.kakao.com/docs/latest/ko/local/dev-guide#address-coord)

In [121]:
def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['x']), float(match_first['y'])

이렇게 만든 addr_to_lat_lon 함수를 이용해 남산의 위경도를 추출하자.

In [122]:
df = addr_to_lat_lon("서울 중구 회현동1가")

In [123]:
df

(126.981204133005, 37.5555892070291)

In [124]:
df[0]

126.981204133005

In [125]:
df[1]

37.5555892070291

#### 3. 지리정보 API를 사용해 위경도에서 주소 추출하기
이번엔 반대로 위경도를 이용해 주소를 추출해보자. 

- 개발가이드: [좌표로 주소 변환](https://developers.kakao.com/docs/latest/ko/local/dev-guide#coord-to-address)

In [126]:
def lat_lon_to_addr(lon,lat):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={longitude}&y={latitude}'.format(longitude=lon,latitude=lat)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address_name']
    return str(match_first)

이렇게 만든 lat_lon_to_addr 함수를 이용해 남산의 위경도를 추출하면 다음과 같이 주소가 출력된다.

In [127]:
lat_lon_to_addr(126.981204133005, 37.5555892070291)

'서울특별시 중구 회현동1가'

In [128]:
map = folium.Map(location=[df[1], df[0]], zoom_start=14)

# for n in crime_anal_raw.index:
#     folium.CircleMarker([crime_anal_raw['lat'][n], crime_anal_raw['lng'][n]],
#                        radius = crime_anal_raw['검거'][n]*10,
#                        color='#3186cc', fill_color='#3186cc').add_to(map)

folium.CircleMarker(
    [df[1], df[0]],
    radius = 10,
    color='#3186cc', 
    fill_color='#3186cc').add_to(map)    
map